In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

movies = pd.read_csv('../../data/ml-1m/movies.dat', sep='::', header=None, names=['MovieID', 'Title', 'Genres'], engine='python', encoding='latin-1')
ratings = pd.read_csv('../../data/ml-1m/ratings.dat', sep='::', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python', encoding='latin-1')

item_user_matrix = ratings.pivot(index='MovieID', columns='UserID', values='Rating')
item_user_matrix = item_user_matrix.fillna(0)

item_similarity = cosine_similarity(item_user_matrix)
item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)

def get_item_recommendations(user_id, item_user_matrix, item_similarity_df, n_recommendations=10):
    user_ratings = item_user_matrix.loc[:, user_id]
    weighted_ratings = item_similarity_df.dot(user_ratings)
    sum_of_similarities = item_similarity_df.dot(user_ratings > 0).replace(0, 1)  # Prevent division by zero
    weighted_ratings = weighted_ratings / sum_of_similarities
    already_rated = user_ratings > 0
    weighted_ratings = weighted_ratings[~already_rated]
    recommendations = weighted_ratings.sort_values(ascending=False).head(n_recommendations)
    recommended_movie_ids = recommendations.index

    return movies[movies['MovieID'].isin(recommended_movie_ids)][['Title', 'Genres']]

print(get_item_recommendations(1, item_user_matrix, item_similarity_df))


                                            Title          Genres
977     Schlafes Bruder (Brother of Sleep) (1995)           Drama
1846  Voyage to the Beginning of the World (1997)           Drama
2274                        Naked Man, The (1998)           Drama
2369                         Outside Ozona (1998)  Drama|Thriller
2776                            White Boys (1999)           Drama
3140                  Loves of Carmen, The (1948)           Drama
3147        Vampyros Lesbos (Las Vampiras) (1970)          Horror
3164                         Smashing Time (1967)          Comedy
3461                    Smoking/No Smoking (1993)          Comedy
3820                            Back Stage (2000)     Documentary
